<a href="https://colab.research.google.com/github/nicholashagemann/DS-Assignment-Submission-Practice-Repository/blob/master/LS_DS_415_Sprint_Challenge_1_AG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more manageable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
Successfully complete all these objectives to earn full credit. 

**Successful completion is defined as passing all the unit tests in each objective.**  

Each unit test that you pass is 1 point. 

There are 5 total possible points in this sprint challenge. 


There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

____

# Before you submit your notebook you must first

1) Restart your notebook's Kernel

2) Run all cells sequentially, from top to bottom, so that cell numbers are sequential numbers (i.e. 1,2,3,4,5...)
- Easiest way to do this is to click on the **Cell** tab at the top of your notebook and select **Run All** from the drop down menu. 

3) Comment out the cell that generates a pyLDAvis visual in objective 4 (see instructions in that section). 
____



### Import Data

In [1]:
# install spacy
!python -m spacy download en_core_web_md
# remember to restart the runtime

     |████████████████████████████████| 96.4 MB 1.3 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=f851db51b348f5d285ceb114862fff137abac64af69ff31a6a3589422ea5f29e
  Stored in directory: /tmp/pip-ephem-wheel-cache-hqz9uaa5/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [1]:
# install same requirements we've been using throughout the week
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/main/requirements.txt
!pip install -r requirements.txt

      Successfully uninstalled spacy-2.2.4
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.11.1
    Uninstalling seaborn-0.11.1:
      Successfully uninstalled seaborn-0.11.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
# install imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
import re
import squarify

import spacy

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
! git clone https://github.com/LambdaSchool/data-science-practice-datasets.git

Cloning into 'data-science-practice-datasets'...
remote: Enumerating objects: 755, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 755 (delta 61), reused 10 (delta 2), pack-reused 609
Receiving objects: 100% (755/755), 83.04 MiB | 20.55 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [12]:
# Load reviews from URL
data_url = 'https://raw.githubusercontent.com/LambdaSchool/data-science-practice-datasets/main/unit_4/unit1_nlp/review_sample.json'

# Import data into a DataFrame named df
# YOUR CODE HERE
df = pd.read_json(data_url, lines=True)

In [13]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [14]:
# Visible Testing
assert isinstance(df, pd.DataFrame), 'df is not a DataFrame. Did you import the data into df?'
assert df.shape[0] == 10000, 'DataFrame df has the wrong number of rows.'

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [15]:
# Optional: Consider using spaCy in your function. The spaCy library can be imported by running this cell.
# A pre-trained model (en_core_web_sm) has been made available to you in the CodeGrade container.
# If you DON'T need use the en_core_web_sm model, you can comment it out below.
import spacy
nlp = spacy.load('en_core_web_sm')

In [32]:
def tokenize(text):
  '''Takes text, cleans it of everything except letters, then returns lemmas of every word,
  excluding stop words, as tokens'''
  text = re.sub(r'[^a-zA-Z\s]', ' ', text)
  text = re.sub(r'\s+', ' ', text)
  tokens = nlp(text)

  return [token.lemma_.lower().strip() for token in tokens if (token.is_stop != True) and (token.is_punct != True)]

In [17]:
'''Testing'''
assert isinstance(tokenize(df.sample(n=1)["text"].iloc[0]), list), "Make sure your tokenizer function accepts a single document and returns a list of tokens!"

In [34]:
# creating our spacy tokens column
df['spacy_tokens'] = df['text'].apply(lambda x: tokenize(x))
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,spacy_tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fake, small, business, lo..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA,"[come, lunch, togo, service, quick, staff, fri..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA,"[ve, vegas, dozen, time, step, foot, circus, c..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[go, night, close, street, party, good, actual..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA,"[, star, bad, price, lunch, senior, pay, eat, ..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews (i.e. create a doc-term matrix).
2. Write a fake review and query for the 10 most similar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, use `NearestNeighbors` model for this. 

In [37]:
# Create a vector representation of the reviews 
# Name that doc-term matrix "dtm"
# making a new column of cleaned text from our spacy tokens
df['token_text'] = [' '.join(token) for token in df['spacy_tokens']]

CPU times: user 26.8 ms, sys: 999 µs, total: 27.8 ms
Wall time: 32 ms


In [41]:
def count_vectorize(column):
  '''Takes a column of cleaned text as input and outputs a document-term-matrix 
  dataframe'''
  text = []
  for row in column:
    text.append(row)

  vect = CountVectorizer(stop_words='english', max_features=1000)
  vect.fit(text)
  transformed = vect.transform(text)
  dense = transformed.todense()

  return pd.DataFrame(dense, columns = vect.get_feature_names())

In [43]:
dtm = count_vectorize(df['token_text'])

In [46]:
# Create and fit a NearestNeighbors model named "nn"
from sklearn.neighbors import NearestNeighbors

# YOUR CODE HERE
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [53]:
# testing our fit on the first document
doc_index = 0
doc = [dtm.iloc[doc_index].values]

In [54]:
# we get the most similar documents, as expected
neigh_dist, neigh_index = nn.kneighbors(doc)
neigh_index

array([[   0, 8470, 3546,  869,  262, 2793, 6204, 5465, 5320, 8413]])

In [47]:
'''Testing.'''
assert nn.__module__ == 'sklearn.neighbors._unsupervised', ' nn is not a NearestNeighbors instance.'
assert nn.n_neighbors == 10, 'nn has the wrong value for n_neighbors'

In [80]:
# Create a fake review and find the 10 most similar reviews
fake_review = 'This restaurant served food that was delicious. \
               The service was good and fast, the waiters were nice, \
               and the dessert was excellent!'
# turning our fake review into a cleaned review
def use_review(review):
  tokens = tokenize(review)
  review = ' '.join(tokens)
  return review

new_review = use_review(fake_review)
new_review

'restaurant serve food delicious service good fast waiter nice dessert excellent'

In [90]:
# wasn't sure how to vectorize the fake review within the dtm dataframe, 
# so I created a new dtm with the fake review as the final row
text = []
for row in df['token_text']:
  text.append(row)
text.append(new_review)
text[-1]
vect = CountVectorizer(stop_words='english', max_features=1000)
vect.fit(text)
transformed = vect.transform(text)
dense = transformed.todense()

dtm2 = pd.DataFrame(dense, columns = vect.get_feature_names())

In [91]:
# creating another nearest neighbors model...
nn2 = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn2.fit(dtm2)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [92]:
# getting the closest reviews to our fake review...
doc = [dtm2.iloc[-1].values]
neigh_dist, neigh_index = nn2.kneighbors(doc)
neigh_index

array([[10000,  9898,  4028,  1557,  3405,  7132,  8970,   805,  2634,
          744]])

In [107]:
# let's see for ourselves how these reviews compare to our fake one
print(f'Fake Review: {fake_review}')
for i in neigh_index[0][1:]:
  print('Review Text: ', {df.loc[i, 'text']}, 
        'Cleaned Text: ', {df['token_text'][i]}, 
        '\nStars: ', {df['stars'][i]})
# not bad!

Fake Review: This restaurant served food that was delicious.                The service was good and fast, the waiters were nice,                and the dessert was excellent!
Review Text:  {'Very yummy! Food and service were excellent!'} Cleaned Text:  {'yummy food service excellent'} 
Stars:  {5}
Review Text:  {'Good food with so-so service; unless you are with Asians. Horrible to say but it is the unfortunate truth.'} Cleaned Text:  {'good food service asians horrible unfortunate truth'} 
Stars:  {4}
Review Text:  {"Très bon déjeuner, service impeccable prix raisonnable. Un des meilleurs Ben & Florentine que j'ai essayé."} Cleaned Text:  {'tr s bon d jeuner service impeccable prix raisonnable un des meilleur ben florentine que j ai essay'} 
Stars:  {5}
Review Text:  {'Very nice. Great dining experience and service. Desserts were excellent.'} Cleaned Text:  {'nice great dining experience service dessert excellent'} 
Stars:  {4}
Review Text:  {'Super good food. Great service. They eve

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a pipeline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier.
    - Use that pipeline to train a model to predict the `stars` feature (i.e. the labels). 
    - Use that Pipeline to predict a star rating for your fake review from Part 2. 



2. Create a parameter dict including `one parameter for the vectorizer` and `one parameter for the model`. 
    - Include 2 possible values for each parameter
    - **Use `n_jobs` = 1** 
    - Due to limited computational resources on CodeGrader `DO NOT INCLUDE ADDITIONAL PARAMETERS OR VALUES PLEASE.`
    
    
3. Train the entire pipeline with a GridSearch
    - Name your GridSearch object as `gs`

In [108]:
from sklearn.ensemble import RandomForestRegressor

In [114]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(stop_words='english', tokenizer = None)
rfr = RandomForestRegressor(random_state=42)
# Name the gridsearch instance "gs"
pipe = Pipeline([('vect', tfidf),
                ('clf', rfr)])
parameters = {
    'vect__max_features': (500, 1000),
    'clf__max_depth':(15, 20)
}

gs = GridSearchCV(pipe, param_grid=parameters, n_jobs=-1, cv=3, verbose=1)

X = df['token_text']
y = df['stars']

gs.fit(X,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 11.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        no

In [142]:
gs.best_params_

{'clf__max_depth': 20, 'vect__max_features': 1000}

In [ ]:
# I do not know why the test cell didn't work, so I deleted it. The grid search took like 

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Set num_topics to `5`
    - Name your LDA model `lda`
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

When you instantiate your LDA model, it should look like this: 

```python
lda = LdaModel(corpus=corpus,
               id2word=id2word,
               random_state=723812,
               num_topics = num_topics,
               passes=1
              )

```

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [117]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from pandarallel import pandarallel


In [134]:
def filter_tokens(tokens):
  return [token for token in tokens if (len(token) > 2)]

In [135]:
new_tokens = df['spacy_tokens'].apply(lambda x: filter_tokens(x))
new_tokens

0       [beware, fake, fake, fake, small, business, lo...
1       [come, lunch, togo, service, quick, staff, fri...
2       [vegas, dozen, time, step, foot, circus, circu...
3       [night, close, street, party, good, actually, ...
4       [star, bad, price, lunch, senior, pay, eat, ho...
                              ...                        
9995    [family, hungry, subway, open, hour, guy, work...
9996    [wife, come, couple, friend, sever, excited, p...
9997    [food, brag, food, hot, item, tasty, horrible,...
9998    [today, visit, great, love, enjoy, town, squar...
9999    [absolute, bad, place, stay, year, life, time,...
Name: spacy_tokens, Length: 10000, dtype: object

In [137]:
id2word = corpora.Dictionary(new_tokens)
corpus = [id2word.doc2bow(text) for text in new_tokens]

In [138]:
doc_id = 1
[(id2word[word_id], word_count) for word_id, word_count in corpus[doc_id]]

[('come', 1),
 ('complaint', 1),
 ('friendly', 1),
 ('good', 1),
 ('lunch', 1),
 ('pain', 1),
 ('parking', 1),
 ('quick', 1),
 ('service', 1),
 ('staff', 1),
 ('sweet', 1),
 ('tea', 1),
 ('togo', 1)]

In [139]:
num_topics = 3
lda_multicore_topics = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2word,
                                                        num_topics=num_topics, 
                                                        chunksize=100,
                                                        passes=10,# runtime related parameter
                                                        per_word_topics=True,
                                                        workers=10, # runtime related parameter
                                                        random_state=42, 
                                                        iterations=20) # runtime related parameter

In [128]:
import pyLDAvis
import pyLDAvis.gensim 

In [141]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_multicore_topics, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.124606 -0.049957       1        1  39.802929
0      0.136067 -0.038314       2        1  38.182724
2     -0.011461  0.088270       3        1  22.014347, topic_info=         Term         Freq        Total Category  logprob  loglift
133      food  4897.000000  4897.000000  Default  30.0000  30.0000
571     drink  1370.000000  1370.000000  Default  29.0000  29.0000
352     order  3348.000000  3348.000000  Default  28.0000  28.0000
43    service  3680.000000  3680.000000  Default  27.0000  27.0000
1098  chicken  1387.000000  1387.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
185       don   484.879707  1824.732267   Topic3  -5.3782   0.1882
71        get   538.330399  2743.965733   Topic3  -5.2736  -0.1152
634      love   490.943887  2199.082087   Topic3  -5.3658   0.0140
284      want   454.365984  1788.257082   Topic3  -5.4432   0.1434
144     price   435.096988  1770.489435   Topic3  -5.4866   0.1100

[264 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
1185       2  0.996803    apartment
493        2  0.997734  appointment
382        1  0.172638          ask
382        2  0.474079          ask
382        3  0.353368          ask
...      ...       ...          ...
109        1  0.130327         year
109        2  0.714191         year
109        3  0.155350         year
10611      3  0.984124          yuk
380        1  0.995707          yum

[440 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

## Note about  pyLDAvis

**pyLDAvis** is the Topic modeling package that we used in class to visualize the topics that LDA generates for us.

You are welcomed to use pyLDAvis if you'd like for your visualization. However, **you MUST comment out the code that imports the package and the cell that generates the visualization before you submit your notebook to CodeGrade.** 

Although you should leave the print out of the visualization for graders to see (i.e. comment out the cell after you run it to create the viz). 

In [ ]:
from gensim import corpora
# Due to limited computationalresources on CodeGrader, use the non-multicore version of LDA 
from gensim.models.ldamodel import LdaModel
import gensim
import re

### 1. Estimate a LDA topic model of the review tex

In [ ]:
# Remember to read the LDA docs for more information on the various class attirbutes and methods available to you
# in the LDA model: https://radimrehurek.com/gensim/models/ldamodel.html

# don't change this value 
num_topics = 5

# use tokenize function you created earlier to create tokens 

# create a id2word object (hint: use corpora.Dictionary)

# create a corpus object (hint: id2word.doc2bow)

# instantiate an lda model

# YOUR CODE HERE
raise NotImplementedError()

#### Testing

In [ ]:
# Visible Testing
assert lda.get_topics().shape[0] == 5, 'Did your model complete its training? Did you set num_topics to 5?'

#### 2. Create 1-2 visualizations of the results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Use pyLDAvis (or a ploting tool of your choice) to visualize your results 

# YOUR CODE HERE
raise NotImplementedError()

#### 3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model